## ДЗ к уроку 1

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_goods = pd.DataFrame([
             [156,  100, 1],
             [1134, 200, 1],
             [27,   150, 1],
             [143,  100, 1],
             [991,  800, 1],
             [1543, 200, 1],
             [3345, 100, 1],
             [533,  300, 1],
             [11,    50, 1],
             [143,    3, 1],
             [521,  550, 0],
             [32,   450, 0],
], columns=['id', 'price', 'is_recommended'])

df_user_bought = pd.DataFrame([
    [ 143, 1 ],
    [ 143, 2 ],
    [ 991, 2 ],
    [ 991, 3 ],
    [ 521, 1 ],
    [ 521, 3 ],
    [ 32,  2 ],
], columns=['gid', 'uid'])


df_recommended     = df_goods.loc[df_goods['is_recommended'] == 1]
df_merge           = pd.merge(df_goods, df_user_bought, left_on=['id'], right_on=['gid'], how='right')
df_bought          = df_merge.drop_duplicates('id') 

recommended_list   = df_recommended['id']
prices_recommended = df_recommended['price']

bought_list        = df_bought['id']
prices_bought      = df_bought['price']

user_bought_list   = list(df_merge.groupby('uid')['gid'].apply(lambda x: list(set(x))))
                                                        

### Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [3]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    hit_rate = int(np.isin(bought_list, recommended_list).sum() > 0)
    
    return hit_rate

In [4]:
for k in [5, 4, 3]:
    print(f"hit rate@{k} = {hit_rate_at_k(recommended_list, bought_list, k)}")

hit rate@5 = 1
hit rate@4 = 1
hit rate@3 = 0


### Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  



In [5]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    flags = np.isin(recommended_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [6]:
for k in [5, 4, 3]:
    print(f"Money precision@{k} = {money_precision_at_k(recommended_list, bought_list, prices_recommended, k):.3f}")

Money precision@5 = 0.667
Money precision@4 = 0.182
Money precision@3 = 0.000


### Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

  

In [7]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    recall = np.isin(bought_list, recommended_list).sum() / len(bought_list)

    return recall


def money_recall_at_k(recommended_list, bought_list, prices_bought, k=5):
# prices_recommended здесь не нужен, так как цены все равно берутся только от релевантных (купленных) товаров
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_bought = np.array(prices_bought)

    flags = np.isin(bought_list, recommended_list)

    recall = np.dot(flags, prices_bought).sum() / prices_bought.sum()

    return recall

In [8]:
for k in [5, 4, 3]:
    print(f"Recall@{k} = {recall_at_k(recommended_list, bought_list, k):.3f}")
    print(f"Money recall@{k} = {money_recall_at_k(recommended_list, bought_list, prices_bought, k):.3f}")

Recall@5 = 0.500
Money recall@5 = 0.474
Recall@4 = 0.250
Money recall@4 = 0.053
Recall@3 = 0.000
Money recall@3 = 0.000


### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [9]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    return np.isin(bought_list, recommended_list).sum() / len(recommended_list)

def ap_at_k_my_fixed(recommended_list, bought_list, k=5): # код скопирован с семинара, с исправлением ошибки (потом заметила новую версию в ДЗ)
    
    bought_list = np.array(bought_list)
    # если мы учитываем top-k рекоммендованных, логично сразу отсекать k, так как это влияет делитель sum(flags) ниже
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(k): # здесь была ошибка, мы должны учитывать элементы, начиная с 0-го ...
        if flags[i]:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1) # ... а вот в precision_at_k надо передавать i+1, чтобы и i-й учелся
            sum_ += p_k
    result = sum_ / sum(flags) # это отличается от формулы в https://habr.com/ru/company/econtenta/blog/303458/
    return result

def ap_at_k(recommended_list, bought_list, k=5): # взято из страницы с ДЗ
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list) # кажется, здесь тоже по хорошему надо сразу отсечь k, так как релевантные рекомендации выше k начинают влиять в делителе ниже

    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    amount_relevant = len(relevant_indexes) 

    relevant_indexes = relevant_indexes[relevant_indexes <= k]

    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant


def map_at_k(recommended_list, bought_list, k=5): # bought_list - список bought_list-ов, например по по всем покупателям
    apks = list(map(lambda ul: ap_at_k(recommended_list, ul, k), list(bought_list)))
    return np.mean(apks)
    

In [10]:
for k in [5, 4, 3]:
    print(f"AP@{k} = {ap_at_k(recommended_list, bought_list, k):.3f}")
    print(f"MAP@{k} = {map_at_k(recommended_list, user_bought_list, k):.3f}")

AP@5 = 0.217
MAP@5 = 0.181
AP@4 = 0.217
MAP@4 = 0.181
AP@3 = 0.083
MAP@3 = 0.069


### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [11]:
def reciprocal_rank_at_k(recommended_list, bought_list, k): # добавила параметр k, так как речь идет про RR@k
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(recommended_list, bought_list)
    found = np.argwhere(flags)
    return 1 / (found[0][0] + 1) if found.shape[0] > 0 else 0 # не найдено - k=inf 
    

def mean_reciprocal_rank_at_k(recommended_list, bought_lists, k):  # bought_list - список bought_list-ов, например по по всем покупателям
    apks = list(map(lambda ul: reciprocal_rank_at_k(recommended_list, ul, k), list(bought_list)))
    return np.mean(apks)


In [12]:
for k in [5, 4, 3]:
    print(f"RR@{k} = {reciprocal_rank_at_k(recommended_list, bought_list, k):.3f}")
    print(f"MRR@{k} = {mean_reciprocal_rank_at_k(recommended_list, user_bought_list, k):.3f}")

RR@5 = 0.250
MRR@5 = 0.113
RR@4 = 0.250
MRR@4 = 0.062
RR@3 = 0.000
MRR@3 = 0.000
